# MetaMagDyn

FMR simulation and analysis software using MuMax3 and Python.

Step by step guide

Import MetaMagDyn as a Package

In [1]:
import metamagdyn
import numpy as np

#Change this if needed 
metamagdyn.runner.mumaxBin = "/usr/bin/mumax3"

Create a .mx3 file containg only the geometry, materials information and initial magnetization  
Use an external editor or use the following template

In [2]:
#Here we create the file from a string
myFileName = 'Test.mx3'
myFileData = '''
//Disk r = 0.5um x 20nm

Nx := 64
Ny := 64
Nz := 1
radius := 0.5e-6
sizeX := radius
sizeY := radius
sizeZ := 20.0e-9
SetGridSize(Nx, Ny, Nz)
SetCellSize(sizeX/Nx, sizeY/Ny, sizeZ/Nz)

setgeom( circle(radius) )
DefRegion(1, circle(radius))

//Material properties
alpha.SetRegion(1, 0.01)
Msat.SetRegion(1, 800e3)
Aex.SetRegion(1, 13e-12)

//Initial magnetization
m.SetRegion(1, Vortex(1,1))
'''
with open(myFileName, 'w') as text_file:
    text_file.write(myFileData)

### SimRunner Class

Initialize a new simulation using a SimRunner

In [3]:
mx3File = 'Test.mx3'
MySim = metamagdyn.SimRunner(mx3File)

Before initilization  
We must fill the fill the following data  

**DC field list**  
    MySim.hs  

**Frequency list**  
    MySim.fs  

**DC field direction**  
    MySim.DC_dir  

**RF field direction**  
    MySim.RF_dir  

In [4]:
MySim.hs = np.linspace(0, 1000, 51)  # from 0 to 1000 Oe (51 points / 20 Oe steps)
MySim.fs = np.linspace(4, 12, 81)*1E9  # from 4 to 12 GHz (81 points / 100 MHz steps)
MySim.DC_dir = [1,0,0] # x direction
MySim.RF_dir = [0,1,0] # y direction

Then we initialize and save the simulation data

In [5]:
SimName = 'Disk_DC_X_RF_Y'
MySim.save(SimName, overwrite=False)

If the simulation was already initialized, you will get a message that *Sim_tables* and *Sim_info* already exist and no action is done. This will protect your already simulated data

For reset the simulation delete the entire folder named as your SimName

To use an already initilized simulation, just create the **SimRunner** using the folder name

In [6]:
SimName = 'Disk_DC_X_RF_Y'
MyRunner = metamagdyn.SimRunner(SimName)

# DC magentization curve

Before any RF simulation, the static magnetic states should be calculated.  
Just run:

In [7]:
SimName = 'Disk_DC_X_RF_Y'
MyRunner = metamagdyn.SimRunner(SimName)
MyRunner.Run_DC_Field()

Check the status of the simulation using:

In [9]:
MyRunner.status
#Wait until Idle

'Idle'

# RF simulation

Whait until the DC_Runner is finished

Just run:

In [14]:
SimName = 'Disk_DC_X_RF_Y'
MyRunner = metamagdyn.SimRunner(SimName)
MyRunner.Run_All_Osc(n_threads = 2)

In [17]:
MyRunner.status
#Wait until Idle or Stop

'Idle'

You can stop the simulation doing:

In [12]:
MyRunner.Stop()

In [18]:
MyRunner.status
#Wait until Idle

'Idle'

In [ ]:
i